# Story Gen Module Setup

## Setup Secrets
Before running this file go to key icon on left hand side and setup secrets for your Azure deployed gpt4o.

Setup keys as given below and **allow notebook access**.
```
AZURE_OPENAI_ENDPOINT = xxxx
AZURE_OPENAI_API_KEY = xxxx
AZURE_OPENAI_API_VERSION = xxx
```

## Drive Permissions
We all know notebooks are not reliable to save the responses from LLM we will save them on drive.

Give your **all** drive permission to this colab notebook.

Master Folder with Kahani will be created in your drive and for each run new folder with timestamp will be created.

In [ ]:
!pip install -q openai
!pip install -q tqdm

import json
import os
from collections import defaultdict
from datetime import datetime
from zoneinfo import ZoneInfo

from google.colab import userdata, drive
from openai import AsyncAzureOpenAI
from openai.types.chat import ChatCompletionMessageParam
from pprint import pprint
from pydantic import BaseModel, Field
from tqdm import tqdm
from typing import List, Optional, Dict

# Environment Variables
AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_KEY = userdata.get('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_API_VERSION = userdata.get('AZURE_OPENAI_API_VERSION')

AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT_GPT4O')
AZURE_OPENAI_API_KEY = userdata.get('AZURE_OPENAI_API_KEY_GPT4O')
AZURE_OPENAI_API_VERSION = userdata.get('AZURE_OPENAI_API_VERSION_GPT4O')


# initialize the client
azure_openai_client = AsyncAzureOpenAI(
            api_key=AZURE_OPENAI_API_KEY,
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION
            )

# azure gpt deployment name
model = "gpt-4o"

ist = ZoneInfo("Asia/Kolkata")
RUN_TIME = datetime.now(ist).strftime("%d-%m-%Y_%H-%M-%S")

def setup_folder():
    """Ask the user for a folder name and set up the folder in Google Drive."""
    drive.mount('/content/drive')
    folder_path = f'/content/drive/My Drive/Kahani/{RUN_TIME}'
    os.makedirs(folder_path, exist_ok=True)
    print(f"Folder created at: {folder_path}")
    return folder_path

def save_file_to_folder(folder_path: str, file_name: str, content: str):
    """Save a file with the given content to the specified folder."""
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"File saved at: {file_path}")

def load_save_files(folder_path: str, file_name: str):
    """ Load saved file in drive"""

    # intial drive location
    drive_location = "/content/drive/My Drive/Kahani"

    # Check if folder exist
    folder_path = f"{drive_location}/{folder_path}"
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return None

    # Check if file exist
    file_path = f"{folder_path}/{file_name}"
    if not os.path.exists(file_path):
        print(f"File '{file_path}' does not exist.")
        return None

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    print(f"File loaded successfully from: {file_path}")
    return content


folder_path = setup_folder()

# Test Run
Once you run this cell, open the drive and go to folder and check if cat.json is stored

In [ ]:
# Testing the API
messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": "You are a cat lover, and you know lot of fascinating facts about cats."},
    {"role": "user", "content": "Can you tell me some interesting facts about ginger cats in bangalore?"},
]

# Cat Schema Response
class CatFactsResponse(BaseModel):
    cat_color: str
    activity_level: str
    behavior: str
    fascinating_facts: str


response = await azure_openai_client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    response_format=CatFactsResponse,
    temperature=1.0
)
parsed_content = response.choices[0].message.content
cat_content = json.loads(parsed_content)
pprint(cat_content)
print("Saving the cool facts about cats in your drive!")
save_file_to_folder(folder_path, "cat.json", parsed_content)

# Source Content

YOU CAN RUN THIS BLOCK WITHOUT INPUT.
- A source content can be a pre-made curriculum or a list of topics and concepts to be included in the story-based learning curriculum. It can be written descriptively for each topic or presented as bullet points. It may also contain all the facts and definitions that the LLM requires to remain factually accurate

In [ ]:
source_content = """

"""

# Defining the Target Audience

*   **`target_audience`**: This variable is intended to hold a detailed description of the specific group of people you want to reach. This could include:
    *   **Demographics:** Age, gender, location, income, education level.
    *   **Psychographics:** Interests, hobbies, values, lifestyle, and attitudes.
    *   **Behavioral Traits:** How they interact with similar products or content.

*   **`looks_and_outfits`**: This variable is designed to define the visual style and aesthetic associated with your target audience. This could involve describing:
    *   **Fashion Choices:** Preferred clothing brands, styles (e.g., vintage, modern, minimalist), and specific articles of clothing.
    *   **Overall Appearance:** Hairstyles, grooming, and general aesthetic.
    *   **Visual Identity:** This helps in creating a consistent and appealing visual language for your project, whether it's for character design, marketing materials, or user interface design.
* **`CITY`**, **`STATE`** and **`COUNTRY`**: Specifying the geographic location of your target audience

By clearly defining these aspects of your target audience, you can create a more focused and effective project that resonates with the right people.

In [ ]:
target_audience = """

"""

looks_and_outfits = """

"""

CITY = ""
STATE = ""
COUNTRY = ""

# Curriculum Development Prompts
YOU CAN RUN THIS BLOCK WITHOUT INPUT
- Here are all the prompts used while developing the curriculum. You can edit the prompts, but do not change the name of the prompts.
- The prompts here are written with a focus on a case study about a Financial Literacy curriculum for low-income rural and semi-urban Indians.


In [ ]:
# @title System Instruction
curriculum_agent_system_instruction = f"""You are a Financial Literacy curriculum developer, focused on developing curricula for low-income rural and semi-urban Indians.

Objective:
- Develop a curriculum with 5 modules to teach financial literacy concepts to low-income Indians.
- The entire curriculum will have an overarching narrative, starting with basic financial concepts and getting progressively harder.
- Each module will focus on one aspect of financial literacy, such as the importance of saving, opening a bank account or loan taking.
- Each module will act as a chapter within the curriculum and contribute to the overarching narrative with connecting threads.
- This curriculum will be used as the basis for creating a narrative that takes a protagonist from being a financial illiterate or a beginner, to becoming financially literate and stable.

About the audience:
{target_audience}

Content Guidelines:
You will use additional information to ground the learning material in real-world schemes, processes, and information provided by banks. Do not make direct references to any specific bank, but ensure that the information remains factual and accurate.
"""

In [ ]:
# @title Generation Instruction
curriculum_agent_generation_instruction = f'''
Generate a 5-module curriculum to teach financial literacy concepts to low-income Indians of vilages in {CITY}, {STATE}, {COUNTRY}. Each module should act as a short story, contributing to an overarching narrative with connecting threads.

Follow the steps below and generate outputs for each step:
1. Define 5 broad learning goals based on the audience and the source material
2. Go through the source material and list important topics to be taught within each learning goal. Be exhaustive with all the topics you can find in the information sheet.
3. Develop an initial curriculum with module names, goals and topics. Include a brief description for each module and topic.
   - Limit each module to a manageable number of topics to ensure each can be developed into a self-contained story with sufficient detail and drama.
   - Ensure progressive complexity: Each module should introduce progressively more complex topics and concepts, ensuring any story based on this curriculum ensures the growth and development of the protagonist in the stories.

The final curriculum should have 10 modules in the following JSON format:

{{
  "curriculum": [
    {{
      "module_number": "Module Number (1)",
      "module_name": "Module 1 Name",
      "goal_of_the_module": "Goal of Module 1",
      "topics_to_be_covered": [
        {{
          "topic_name": "Name of Topic 1.1",
          "description": "Description of Topic 1.1"
        }},
        ...
        {{
          "topic_name": "Name of Topic 1.n",
          "description": "Description of Topic 1.n"
        }}
      ]
    }},
    ...
    {{
      "module_number": "Module Number (5)",
      "module_name": "Module 5 Name",
      "goal_of_the_module": "Goal of Module 5",
      "topics_to_be_covered": [
        {{
          "topic_name": "Name of Topic 5.1",
          "description": "Description of Topic 5.1"
        }},
        ...
        {{
          "topic_name": "Name of Topic 5.n",
          "description": "Description of Topic 5.n"
        }}
      ]
    }}
  ]
}}
'''

In [ ]:
# @title Evaluation Instruction
curriculum_agent_evaluation_instruction= """
Along with the user feedback, evaluate the generated curriculum based on the following criteria:

1. **Appropriateness for Target Audience**:
   - Evaluate the complexity of the curriculum. Is it appropriate for low-income Indians who are new to financial concepts?
   - Are the topics relevant and useful for the target audience?
   - Are the topics relatable to the daily experiences and challenges of the target audience?

2. **Accuracy Based on the Information Sheet**:
   - Cross-check the curriculum against the source documents for factual accuracy.
   - Ensure that the information provided is correct and follows the guidelines provided in the information sheet.

3. **Progressive Complexity and Story Integration**:
   - Ensure that each module has an appropriate number of topics and that these topics can be integrated into a self-contained story.
   - Verify that the modules introduce progressively more complex topics and concepts.

4. **Order and Prerequisite Information**:
   - Ensure that all prerequisite information for a module is covered in previous modules.
   - Verify that each module builds on the information taught in the previous modules.

For each criterion, list down problems you see in the initial material based on the evaluation and suggest concrete changes explicitly referencing the initial module.

Provide the evaluation report in the following format:

Evaluation based on User Feedback:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5

Evaluation based on Criterion 1:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5

Evaluation based on Criterion 2:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5

...

Evaluation based on Criterion 4:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5
"""

In [ ]:
# @title Final Instruction
curriculum_final_instruction= """
Based on the points mentioned in the evaluation, edit the final curriculum and present it in the following JSON format:

{
  "curriculum": [
    {
      "module_number": "Module Number (1)",
      "module_name": "Module 1 Name",
      "goal_of_the_module": "Goal of Module 1",
      "topics_to_be_covered": [
        {
          "topic_name": "Name of Topic 1.1",
          "description": "Description of Topic 1.1"
        },
        ...
        {
          "topic_name": "Name of Topic 1.n",
          "description": "Description of Topic 1.n"
        }
      ]
    },
    ...
    {
      "module_number": "Module Number (5)",
      "module_name": "Module 5 Name",
      "goal_of_the_module": "Goal of Module 5",
      "topics_to_be_covered": [
        {
          "topic_name": "Name of Topic 5.1",
          "description": "Description of Topic 5.1"
        },
        ...
        {
          "topic_name": "Name of Topic 5.n",
          "description": "Description of Topic 5.n"
        }
      ]
    }
  ]
}
"""

# Curriculum Development Agents
Here you will run the code to generate the curriculum in the following steps:
1. Generate a draft
2. Give your feedback on the draft [INPUT REQUIRED]
3. Evaluate the draft
4. Create a final json of the curriculum incorporating your feedback and the results of the evaluation
5. Parse the final json into plain text
6. Save the plain text to your drive

In [ ]:
# @title Generate Curriculum Draft
user_prompt = curriculum_agent_generation_instruction + f"\nUse the following information sheet for reference:\n{source_content}"

messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": curriculum_agent_system_instruction},
    {"role": "user", "content": user_prompt},
]


response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=1.0
)
curriculum = response.choices[0].message.content
print(curriculum)

In [ ]:
# @title Users Feedback
curriculum_agent_feedback = input("Enter the feedback on curriculum ")
if curriculum_agent_feedback == "":
  curriculum_agent_feedback = "No user's feedback everything looks good!"


In [ ]:
# @title Curriculum Evaluator
# Generate a detailed evaluation report based on the provided feedback
# This step analyzes the curriculum against our specified criteria

# Evaluation Prompt
messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": "Come up with an evaluation report based on user feedback and evaluation criteria"},
    {"role": "user", "content": f"User feedback:\n{curriculum_agent_feedback}\n {curriculum_agent_evaluation_instruction}"},
]

# Send the evaluation request to the model
# The chat context includes our initial curriculum and all previous context
evaluation_response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=1.0
)
curriculum_evaluation_report = evaluation_response.choices[0].message.content

# Display the evaluation report for review
print("\nEvaluation Report Generated:\n", curriculum_evaluation_report)

In [ ]:
# @title Final Curriculum Structure
# Generate the final version of the curriculum
# This step incorporates all feedback and evaluation insights

messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": curriculum_final_instruction},
    {"role": "user", "content": f"{curriculum_evaluation_report}"}
]


final_response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=1.0
)

final_curriculum = final_response.choices[0].message.content
print(final_curriculum)

In [ ]:
# @title Curriculum Parser

def parse_curriculum(json_string):
    # Extract the JSON string from the text
    start = json_string.find('{')
    end = json_string.rfind('}') + 1
    json_data = json_string[start:end]
    # print(json_data)

    # Parse the JSON data
    data = json.loads(json_data)


    # Extract the curriculum
    curriculum = data['curriculum']

    # Create a string to store the parsed information
    output = ""

    # Format the parsed information
    for module in curriculum:
        output += f"Module {module['module_number']}: {module['module_name']}\n"
        output += f"Goal: {module['goal_of_the_module']}\n"
        output += "Topics:\n"
        for topic in module['topics_to_be_covered']:
            output += f"  - {topic['topic_name']}: {topic['description']}\n"
        output += "\n"

    return output

parsed_curriculum = parse_curriculum(curriculum)

# Print the parsed curriculum
print(parsed_curriculum)

In [ ]:
# @title Saving Curriculum
print("Saving Curriculum to your drive....")
save_file_to_folder(folder_path, "curriculum.txt", parsed_curriculum)

In [ ]:
# Load the file code goes here - function is created above
# !TODO: Implementation
parsed_curriculum = load_save_files(RUN_TIME, "curriculum.txt")

# Narrative Development Prompts
YOU CAN RUN THIS BLOCK WITHOUT INPUT
- Here are all the prompts used while developing the narrative. You can edit the prompts, but do not change the name of the prompts.
- We are generating a five-module, story-based Financial Literacy curriculum for low-income Indians
- Here, we will be generating five different narrative approaches for the five-module story. You can modify the number according to your preference. 
- By default, six diverse characters are generated for the story. You can modify this number as needed.
- India-specific examples are included; please adapt or rewrite them to suit your target country.

In [ ]:
# @title System Instruction
narrative_system = f"""
You are a Financial Literacy story-based curriculum developer specializing in teaching through storytelling. Your task is to create a 5-module story-based financial literacy curriculum that teaches financial literacy concepts to low-income Indians through compelling narratives, featuring well-developed characters. The curriculum should feel like a single, continuous story of the protagonist's journey, with each module serving as an 'episode' in a series.

1.Your Audience:
  {target_audience}

2.Source Materials:
  You have two resources to guide your curriculum development:
  **Source File**:
    This information sheet provides insights into financial products and services commonly offered by banks in India.
    Use this information to create realistic scenarios and details within the stories you generate.
    For example, when describing opening a bank account, incorporate the details about documentation, account types, and potential benefits mentioned in the information sheet.
    Use the information sheet as a reference to ensure accuracy and authenticity in the financial details presented within the stories.
    Remember, your goal is to weave these details seamlessly into the narrative to enhance the learning experience without directly mentioning any particular bank, financial schemes or .
  **'Learning Curriculum'**:
    This file contains the 5-module learning curriculum outlining the financial literacy concepts you need to teach.
    Use each module's 'Goal of the Module' and 'Topics to be Covered' sections as inspiration to create the story arcs for each module.
    For example, if a module focuses on 'Savings and Investment Options,' your story arc should involve the protagonist learning about and applying different savings and investment strategies.
    Remember, the stories you create should teach the concepts presented in the learning curriculum in an engaging and relatable way.

3.Cultural Context of the Stories:
  You will follow Newmark’s five markers of cultural words to make sure the stories are culturally relevant:
  A. Ecological Elements:
    - local geography, climate, and natural landmarks specific to the region
    - indigenous flora and fauna
    - seasonal changes and their impact on daily life
    - local terrain features that shape the community's lifestyle
  B. Material Culture:
    - regional food, ingredients, and traditional dishes
    - character clothing, from everyday wear to ceremonial attire
    - traditional architecture and housing styles
    - local modes of transportation and tools
    - traditional crafts, artifacts, and artistic practices
  C. Social Culture:
    - leisure activities and games unique to the community
    - work practices and occupational traditions
    - characters from different socio-economic backgrounds
    - how social class and caste influence daily interactions
    - communal gathering practices
  D. Organizations, Customs, and Concepts:
    - religious practices, festivals, and ceremonies
    - local political and administrative systems
    - educational customs, norms and practices
    - artistic and cultural organizations' roles in the community
    - traditional ceremonies and rituals
  E. Gestures and Habits:
    - culturally specific body language and gestures
    - proper greetings and respect customs
    - local etiquette and social norms
    - traditional ways of showing emotion
    - culturally appropriate dialogue patterns and expressions


4.Tone and Style:
  - The vocabulary you use will be simple, and you will avoid phrases, words or other grammatical instruments that are difficult to translate to indic languages.
  - Use third person narrative and first person dialogue.
  - The tone of your writing should be narrative fiction.
  - Keep your sentences short.
  - Ensure that the narrative is filled with high drama and soap opera-like elements. Include dramatic conflicts, unexpected twists, and intense emotional scenes to keep the audience engaged.

*Reiterating your role and objective*: Your role is to develop a 5-module story-based financial literacy curriculum that is educational, engaging, and relatable for low-income Indians. The objective is to create a cohesive and continuous narrative that effectively teaches financial literacy concepts through well-developed characters and compelling stories with high drama and soap opera-like elements. The curriculum should feel like a single, continuous story of the protagonist's journey, with each module serving as an 'episode' in a series.
"""

In [ ]:
# @title Narrative Option Generation Instruction
narrative_options = """
Generate 5 different narrative approaches for the 5-module story-based financial literacy curriculum. Each narrative should outline a unique approach to how the protagonist learns financial literacy concepts and grows as a character.

For each narrative approach, provide the following details:

**Logline**: A brief description of the main plot and unique approach of the narrative. The plot should be filled with drama and interesting events.
**Protagonist's Learning Journey**: Explain how the protagonist learns financial literacy concepts through the narrative.
**Character Development**: Describe the growth and development of the protagonist throughout the story, considering their cultural background, values, and social norms.
**High Drama Elements**: Highlight dramatic conflicts, unexpected twists, and intense emotional scenes in the narrative, making sure they are culturally relevant and resonant.
**Cultural Elements**: Detail how the narrative incorporates cultural elements such as location, landmarks, scenery, diverse characters, art, food, social norms, values, religious festivals, and cultural artifacts.

Ensure that each narrative approach:
Presents a unique storyline, in which the protagonist learns financial literacy through different methods or experiences.
Demonstrates the protagonist's journey and development in a culturally relevant and relatable manner.
Seamlessly integrates financial literacy concepts into the story, enriched with cultural elements.
Includes high drama and soap opera-like elements to keep the narrative engaging and captivating.
Focus on giving the protagonists diverse ages, genders, religions and castes, occupations, and socio-economic backgrounds. Ensure their clothing, language, and behavior are representative of their cultural and social norms.


Use the learning curriculum and information sheet for reference.
"""

In [ ]:
# @title Character Generation Instruction

### Character Development with single JSON Output

character_generation= """
Based on the chosen narrative, flesh out the characters mentioned and any other supporting characters the story could need. Develop a maximum of 6 characters.

**Create 6 Diverse Characters**:
  1. Assign each character a primary role to enhance the narrative:
    a. **Protagonist**: The character who drives the story forward in pursuit of the main goal.
    b. **Antagonist**: The character who opposes the protagonist and creates obstacles.
    c. **Guardian**: A mentor figure who guides and aids the protagonist.
    d. **Contagonist**: A character who hinders the protagonist's progress by representing an alternative point of view or solution.
    e. **Sidekick**: A loyal companion who supports the protagonist.
    f. **Skeptic**: A character who questions the protagonist's actions and beliefs.
  2. Design characters with different backgrounds, ages, genders, religions and castes, occupations, and socio-economic situations that are representative of your target audience.
  3. Develop their personalities, relationships with each other, motivations, and cultural backgrounds, including their clothing, language, and religious practices.
  4. Include a central protagonist who features in all modules, facing and learning from each financial challenge. Other characters can have varying levels of involvement throughout the modules.
  5. Consider how each character will contribute to the overarching narrative and how they might change or learn throughout the story.
  6. Enrich character profiles with details about their cultural practices, social norms, and any religious or festival-related activities they participate in.
  7. Ensure characters’ names appropriately match their religious, cultural and geographical identities.

### Outputs:
Provide the following outputs in a JSON format:

Overarching Character Profiles:
```json
{
  "character_profiles": [
    {
      "character_number": 1,
      "character_role": "Assign one of the defined character roles",
      "name": "Reflecting common names in India",
      "age": "Character Age",
      "occupation": "Character Occupation",
      "financial_personality": "Key traits and how they might influence financial decisions",
      "cultural_background": "Details about the character's region, language, religion, and traditional practices.",
      "clothing_and_appearance": "Description of typical clothing and physical appearance, reflecting cultural norms.",
      "brief_backstory": "Highlighting their relationship to financial matters, including any relevant cultural practices.",
      "relationship_to_other_characters": "Description of relationships with other characters, considering social and cultural norms.",
      "potential_for_growth_change": "How might this character evolve through the story, particularly in relation to cultural and social influences?"
    },
    ...
    {
      "character_number": 6,
      "character_role": "Assign one of the defined character roles",
      "name": "Reflecting common names in India",
      "age": "Character Age",
      "occupation": "Character Occupation",
      "financial_personality": "Key traits and how they might influence financial decisions",
      "cultural_background": "Details about the character's region, language, religion, and traditional practices.",
      "clothing_and_appearance": "Description of typical clothing and physical appearance, reflecting cultural norms.",
      "brief_backstory": "Highlighting their relationship to financial matters, including any relevant cultural practices.",
      "relationship_to_other_characters": "Description of relationships with other characters, considering social and cultural norms.",
      "potential_for_growth_change": "How might this character evolve through the story, particularly in relation to cultural and social influences?"
    }
  ]
}
"""

In [ ]:
# @title Story Arc Development Instruction

### Story Arc Development

story_arc= """
Based on the narrative chosen and the characters developed, develop a story arc for each of the curriculum’s 5 modules.

**Develop Story Arcs for Each Module**:
  - Display each module's "Goal of the Module" and "Topics to be Covered" (including subtopics) sections from the learning curriculum file as is.
  - For each of the 5 modules in the 'Learning Curriculum', craft a distinct story arc, ensuring it flows naturally from the events of the previous module.
  - Each story arc should clearly relate to the module's "Goal of the Module", cover its key topics, and maintain the continuity of the overarching narrative.
  - Leverage the defined character roles to create compelling conflicts, challenges, and transformations within each module's story.
  - Based on Newmark's 5 cultural markers include culturally significant narrative elements, dialogue, items, events, descriptions, and plot points that resonate with the audience’s linguistic and cultural practices.
  - Ensure that the cultural elements are naturally woven into the narrative, without them seeming out of place.
  - Ensure each module includes high drama and soap opera-like elements to keep the narrative engaging and captivating.

  ### Outputs:
  Provide the following outputs in a JSON format:

  Module Arcs:
  ```json
{
  "module_story_arcs": [
    {
      "module_number": 1,
      "goal_of_the_module": "Goal of the Module",
      "topics_to_be_covered": [
        {
          "topic": "Topic Name",
          "description": "Topic Description"
        }
        ...
        {
          "topic": "Topic Name",
          "description": "Topic Description"
        }
      ],
      "featured_characters": {
        "Character Name 1": "Role",
        ...
        "Character Name n": "Role"
      },
      "recap_of_previous_module": "Brief summary of the key events and outcomes of the previous module",
      "high_drama_elements": {
        "conflict": "Describe the main conflict in this module, ensuring it reflects cultural norms and social pressures.",
        "twists": "Detail any unexpected twists that add culturally relevant drama to the story.",
        "emotional_scenes": "Highlight the intense emotional scenes that occur in this module, particularly those tied to cultural values and social norms."
      },
      "turning_learning_objectives_into_story_points": [
        {
          "topic": "Topic Name",
          "role_in_story": "Explain what role the topic will play in the story, considering cultural and social influences."
        }
      ],
      "story_arc": {
        "premise": "Briefly set the scene, including cultural context such as location, landmarks, and significant social or religious events.",
        "triggering_event": "What event occurs that presents the main financial challenge or opportunity related to this module's learning goal?",
        "rising_action": [
          "Describe a sequence of 3-4 key events that create conflict, tension, or decision points for the characters, incorporating cultural elements like social norms, family obligations, or festival-related pressures."
        ],
        "climax": "What is the most dramatic or pivotal moment in the story where the characters face a significant choice or consequence related to the financial literacy concept, influenced by cultural factors?",
        "falling_action": "How do the events immediately following the climax unfold? How do characters react to the consequences of the choices made, particularly in relation to their cultural or social context?",
        "resolution": "What is the outcome of the story? How is the financial challenge addressed or resolved (or not)? What lessons have the characters learned about the financial concept, and how does this relate to their cultural values?",
        "connecting_thread": "Briefly explain how this module's story arc connects to the previous one and creates a natural lead-in to the next, considering cultural continuity."
      }
    },
    ...
    {
      "module_number": 5,
      "goal_of_the_module": "Goal of the Module",
      "topics_to_be_covered": [
        {
          "topic": "Topic Name",
          "description": "Topic Description"
        }
        ...
        {
          "topic": "Topic Name",
          "description": "Topic Description"
        }
      ],
      "featured_characters": {
        "Character Name 1": "Role",
        ...
        "Character Name n": "Role"
      },
      "recap_of_previous_module": "Brief summary of the key events and outcomes of the previous module",
      "high_drama_elements": {
        "conflict": "Describe the main conflict in this module, ensuring it reflects cultural norms and social pressures.",
        "twists": "Detail any unexpected twists that add culturally relevant drama to the story.",
        "emotional_scenes": "Highlight the intense emotional scenes that occur in this module, particularly those tied to cultural values and social norms."
      },
      "turning_learning_objectives_into_story_points": [
        {
          "topic": "Topic Name",
          "role_in_story": "Explain what role the topic will play in the story, considering cultural and social influences."
        }
      ],
      "story_arc": {
        "premise": "Briefly set the scene, including cultural context such as location, landmarks, and significant social or religious events.",
        "triggering_event": "What event occurs that presents the main financial challenge or opportunity related to this module's learning goal?",
        "rising_action": [
          "Describe a sequence of 3-4 key events that create conflict, tension, or decision points for the characters, incorporating cultural elements like social norms, family obligations, or festival-related pressures."
        ],
        "climax": "What is the most dramatic or pivotal moment in the story where the characters face a significant choice or consequence related to the financial literacy concept, influenced by cultural factors?",
        "falling_action": "How do the events immediately following the climax unfold? How do characters react to the consequences of the choices made, particularly in relation to their cultural or social context?",
        "resolution": "What is the outcome of the story? How is the financial challenge addressed or resolved (or not)? What lessons have the characters learned about the financial concept, and how does this relate to their cultural values?",
        "connecting_thread": "Briefly explain how this module's story arc connects to the previous one and creates a natural lead-in to the next, considering cultural continuity."
      }
    }
  ]
}
  ```
  """


In [ ]:
# @title Evaluation Instruction
Narrative_evaluation_instruction = f"""

Along with the user feedback, evaluate the generated curriculum based on the following criteria:

1. **Appropriateness for Audience**:
Evaluate the details of the narrative. Is it appropriate for low-income Indians who are new to financial concepts?
Are the narratives appropriate for a low-income rural indian community who may not have experience with banks or formal financial systems?
Will the narratives resonate with the lives and challenges of low-income individuals working in the informal sector such as daily wage labourers, street vendors and domestic workers?
Are the topics relevant and useful for the target audience?
Are the examples and scenarios relatable to the daily experiences and challenges of the target audience?
Are the characters culturally appropriate for the target audience in {CITY}, {STATE},{COUNTRY}?

2. **Accuracy Based on the Source Material**
Are the details and scenarios within the narratives realistic?
Do the financial details such as opening a bank account, account types and documentation detail presented within the narratives match with the source document?
Are these financial details weaved seamlessly into the narratives?
Do the narratives avoid mentioning a particular bank or financial institutions?

3. **Accuracy based on Curriculum”
Do the narratives reference the curriculum generated?
Do the narratives use the ‘goal of the module’ and ‘topics to be covered’ sections as inspiration to create the story arcs for each module?
For example, if a module focuses on 'Savings and Investment Options,' does the story arc involve the protagonist learning about and applying different savings and investment strategies?
Do the narratives teach the concepts presented in the curriculum in an engaging and relatable way?


4. **Accuracy of Character Development**
Are the characters diverse?
Are the characters representative of the target audience?
Are the characters’ interpersonal relationships, motivations and cultural backgrounds well developed?
Do the narratives consider how each character will contribute to the overarching narrative and how they might change or learn throughout the story?
Is there a central protagonist who features in each module?
Do the characters’ names appropriately match their gender, religious, cultural and geographical identities?

5. **Accuracy of Story Arcs**
Do each of the modules have a distinct story arc?
Do the story arcs of each module flow naturally from the events of the previous module?
Do the story arcs include culturally relevant settings?
Does each story arc clearly relate to the modules “Goal of the Module”, cover its key topics and maintain continuity of the overarching narrative?
Do the story arcs leverage the defined character roles to create compelling conflicts, challenges and transformation within each module's story?
Are the narratives filled with high drama and soap opera-like elements?
Do the narratives include dramatic conflicts, unexpected twists and intense emotional scenes to keep the audience engaged?


For each criterion, list down problems you see in the initial material based on the evaluation and suggest concrete changes explicitly referencing the initial module.

Provide the evaluation report in the following format:

Evaluation based on User Feedback:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5

Evaluation based on Criterion 1:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5

Evaluation based on Criterion 2:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5

...

Evaluation based on Criterion 5:
  Problems:
    Module 1:
      ...
    Module 5:
  Suggested Changes:
    Module 1:
      ...
    Module 5
"""

In [ ]:
# @title Final Instruction
Narrative_final_instruction= f'''
Based on the feedback, come up with the final narrative with 5 modules in the following JSON format
Overarching Narrative and Character Profiles:
“””
{{
    "character_number": 1,  # Unique identifier for the character
    "character_role": "string",  # Assign one of the defined character roles, e.g., "Protaganist", "Antagonist"
    "name": "string",  # Reflecting common names in India, e.g., "Rajesh Sharma", "Priya Patel"
    "age": "string",  # Age , e.g., "30", "13" Age should reflect in the relations with other character. for example, a father at a age of 40 can have a daughter of age 20.
    "hometown": "string",  # Place of origin in/near {CITY}, within the state of {STATE}
    "state": "string",  # {STATE} (Keep it {STATE} Unless Specified)
    "ethnicity": "string",  # In relevance to the story and state, considering social and cultural norms.
    "physical_appearance": {{
        "key_traits": "string",  # Key descriptors, e.g., "strong build, confident"
        "features": "string",  # Notable physical features, e.g., "sharp nose, deep-set eyes"
        "personality": "string",  # Personality traits, e.g., "hardworking, resilient"
        "gender": "string",  # Gender identity, e.g., "male", "female"
        "height": "string",  # Height or description, e.g., "average", "tall"
        "build": "string",  # Body type, e.g., "slim", "stocky", "muscular"
        "hair_type": "string",  # Hair texture, e.g., "thick, wavy", "curly"
        "hair_style": "string",  # Hairstyle, e.g., "neatly combed", "braided"
        "hair_color": "string",  # Hair color, e.g., "black", "dark brown"
        "eye_color": "string",  # Eye color, e.g., "dark brown", "black"
        "skin_tone": "string",  # Skin tone, e.g., "wheatish", "dusky", "fair"
        "facial_shape": "string",  # Face shape, e.g., "oval", "round"
        "nose": "string",  # Nose shape, e.g., "aquiline", "broad"
        "lips": "string",  # Lip type, e.g., "thin", "full"
        "eyebrows": "string",  # Eyebrow shape, e.g., "thick", "arched"
        "posture": "string"  # Posture, e.g., "upright", "relaxed"
    }},
    Description of character outfit that suits the story, reflecting the cultural norms of the story location. The clothing should be appropriate to the age and gender of the character. Specify its color, and type.
    **Keep the outfits simple, practical, and minimalistic. Avoid mentioning an extensive range of colors, intricate embroidery, heavy ornamentation, or luxury textiles.**
    Use clear and concise language to describe garments like sarees, dhotis, lungis, salwar kameez, kurtas, and simple blouses, as well as how they are typically worn. Emphasize the functional aspects of the clothing, such as comfort in hot weather, ease of movement for agricultural work, and cultural significance in daily life.
    "outfit": {{
        "outfit_color": "string",  # Dominant clothing color, e.g., "white", "bright yellow"
        "outfit_type": "string",  # Type of attire that suits the story location, reflecting the cultural norms. The attire should be appropriate to the age and gender of the character., e.g., "kurta-dhoti", "salwar-kameez", "shirt and trousers"
        "patterns_and_design": "string",  # Simple and minimalistic pattern or design, e.g., "floral", "block print"
        "texture_and_fabric": "string",  # Fabric type, e.g., "cotton", "khadi"
        "footwear": "string"  # Footwear type, e.g., "chappals", "mojari", "formal shoes"
    }},
    Add jewelry and its description if it suits with outfit of the character and the context of the story. Add the specific color and type of jewelry.
    Jewelry should be minimal and limited to essential items, such as simple bangles, small earrings, or basic nose studs, as characters from rural India would not have access to or be able to afford elaborate or decorative accessories.
    "jewelry_details": {{
        "jewelry_type": "string",  # Type of jewelry, e.g., "bangles", "mangalsutra", "nose ring"
        "jewelry_color": "string"  # Jewelry color, e.g., "gold", "silver", "brass", "iron"
        # Not all characters need to wear jewelry; this field can be left empty or set to "N/A".
    }},
    Accessories should be minimal and limited to practical, essential items that support daily life in rural India. Include only items such as a small backpack, a simple sickle, a basic stick, or other functional tools relevant to the local lifestyle. Avoid any decorative, ornamental, or luxury accessories.
    "accessory_details": {{
        "accessory_type": "string",  # Accessory type, e.g., "gamcha", "pagdi", "dupatta", "backpack", "purse"
        "accessory_color": "string"  # Accessory color, e.g., "red", "green" simple block colors
        # Not all characters need accessories; this field can also be left empty or set to "N/A".
    }},
    "occupation": "string",  # Profession or livelihood, e.g., "farmer", "teacher", "artisan"
    "financial_personality": "string",  # Key traits and how they might influence financial decisions, e.g., "Traditional and cautious, often wary of modern banking"
    "cultural_background": "string",  # Cultural heritage. Details about the character"s region, language, religion, and traditional practices., e.g., "Comes from a conservative Brahmin family where traditional values are emphasized, and she"s accustomed to community life with rich customs and festivals"
    "brief_backstory": "string",  # Highlighting their relationship to financial matters, including any relevant cultural practices., e.g., "Widowed at a young age, Priya lives with her son and faces societal pressures and financial instability."
    "relationship_to_other_characters": "string",  # Description of relationships with other characters, considering social and cultural norms., e.g., "younger sibling of the protagonist", "mentor of the main character"
    "potential_for_growth_change": "string"  # How might this character evolve through the story, particularly in relation to cultural and social influences, e.g., "Has great potential to transition from a traditional homemaker to a financially empowered individual and community leader by embracing modern finance principles and teaching others."
    }}
    ...
   {{
    "character_number": n,  # Unique identifier for the character
    "character_role": "string",  # Assign one of the defined character roles, e.g., "Protaganist", "Antagonist"
    "name": "string",  # Reflecting common names in India, e.g., "Rajesh Sharma", "Priya Patel"
    "age": "string",  # Age , e.g., "30", "13" Age should reflect in the relations with other character. for example, a father at a age of 40 can have a daughter of age 20.
    "hometown": "string",  # Place of origin in/near {CITY}, within the state of {STATE}
    "state": "string",  # {STATE} (Keep it {STATE} Unless Specified)
    "ethnicity": "string",  # In relevance to the story and state, considering social and cultural norms.
    "physical_appearance": {{
        "key_traits": "string",  # Key descriptors, e.g., "strong build, confident"
        "features": "string",  # Notable physical features, e.g., "sharp nose, deep-set eyes"
        "personality": "string",  # Personality traits, e.g., "hardworking, resilient"
        "gender": "string",  # Gender identity, e.g., "male", "female"
        "height": "string",  # Height or description, e.g., "average", "tall"
        "build": "string",  # Body type, e.g., "slim", "stocky", "muscular"
        "hair_type": "string",  # Hair texture, e.g., "thick, wavy", "curly"
        "hair_style": "string",  # Hairstyle, e.g., "neatly combed", "braided"
        "hair_color": "string",  # Hair color, e.g., "black", "dark brown"
        "eye_color": "string",  # Eye color, e.g., "dark brown", "black"
        "skin_tone": "string",  # Skin tone, e.g., "brown", "dusky", "wheatish", "dark brown"
        "facial_shape": "string",  # Face shape, e.g., "oval", "round"
        "nose": "string",  # Nose shape, e.g., "aquiline", "broad"
        "lips": "string",  # Lip type, e.g., "thin", "full"
        "eyebrows": "string",  # Eyebrow shape, e.g., "thick", "arched"
        "posture": "string"  # Posture, e.g., "upright", "relaxed"
    }},
    Description of character outfit that suits the story, reflecting the cultural norms of the story location. The clothing should be appropriate to the age and gender of the character. Specify its color, and type.
    **Keep the outfits simple, practical, and minimalistic. Avoid mentioning an extensive range of colors, intricate embroidery, heavy ornamentation, or luxury textiles.**
    Use clear and concise language to describe garments like sarees, dhotis, lungis, salwar kameez, kurtas, and simple blouses, as well as how they are typically worn. Emphasize the functional aspects of the clothing, such as comfort in hot weather, ease of movement for agricultural work, and cultural significance in daily life.
    "outfit": {{
        "outfit_color": "string",  # Dominant clothing color, e.g., "black and white", "bright yellow"
        "outfit_type": "string",  # Type of outfit that suits the story location, reflecting the cultural norms. The attire should be appropriate to the age and gender of the character., e.g., "kurta-dhoti", "salwar-kameez", "shirt and trousers"
        "texture_and_fabric": "string",  # Fabric type, e.g., "cotton", "khadi"
        "footwear": "string"  # Footwear type, e.g., "chappals", "mojari", "formal shoes"
    }},
    Add jewelry and its description if it suits with outfit of the character and the context of the story. Add the specific color and type of jewelry.
    Jewelry should be minimal and limited to essential items, such as simple bangles, small earrings, or basic nose studs, as characters from rural India would not have access to or be able to afford elaborate or decorative accessories.
    "jewelry_details": {{
        "jewelry_type": "string",  # Type of jewelry, e.g., "bangles", "mangalsutra", "nose ring"
        "jewelry_color": "string"  # Jewelry color, e.g., "gold", "silver", "brass", "iron"
        # Not all characters need to wear jewelry; this field can be left empty or set to "N/A".
    }},
    Accessories should be minimal and limited to practical, essential items that support daily life in rural India. Include only items such as a small backpack, a simple sickle, a basic stick, or other functional tools relevant to the local lifestyle. Avoid any decorative, ornamental, or luxury accessories.
    "accessory_details": {{
        "accessory_type": "string",  # Accessory type, e.g., "gamcha", "pagdi", "dupatta", "backpack", "purse"
        "accessory_color": "string"  # Accessory color, e.g., "red", "green" simple block colors
        # Not all characters need accessories; this field can also be left empty or set to "N/A".
    }},
    "occupation": "string",  # Profession or livelihood, e.g., "farmer", "teacher", "artisan"
    "financial_personality": "string",  # Key traits and how they might influence financial decisions, e.g., "Traditional and cautious, often wary of modern banking"
    "cultural_background": "string",  # Cultural heritage. Details about the character"s region, language, religion, and traditional practices., e.g., "Comes from a conservative Brahmin family where traditional values are emphasized, and she"s accustomed to community life with rich customs and festivals"
    "brief_backstory": "string",  # Highlighting their relationship to financial matters, including any relevant cultural practices., e.g., "Widowed at a young age, Priya lives with her son and faces societal pressures and financial instability."
    "relationship_to_other_characters": "string",  # Description of relationships with other characters, considering social and cultural norms., e.g., "younger sibling of the protagonist", "mentor of the main character"
    "potential_for_growth_change": "string"  # How might this character evolve through the story, particularly in relation to cultural and social influences, e.g., "Has great potential to transition from a traditional homemaker to a financially empowered individual and community leader by embracing modern finance principles and teaching others."
}}

```

Module Arcs:
```json
{{
  "module_story_arcs": [
    {{
      "module_number": 1,
      "goal_of_the_module": "Goal of the Module",
      "topics_to_be_covered": [
        {{
          "topic": "Topic Name",
          "description": "Topic Description"
        }}
        ...
        {{
          "topic": "Topic Name",
          "description": "Topic Description"
        }}
      ],
      "featured_characters": {{
        "Character Name 1": "Role",
        ...
        "Character Name n": "Role"
      }},
      "recap_of_previous_module": "Brief summary of the key events and outcomes of the previous module",
      "high_drama_elements": {{
        "conflict": "Describe the main conflict in this module, ensuring it reflects cultural norms and social pressures.",
        "twists": "Detail any unexpected twists that add culturally relevant drama to the story.",
        "emotional_scenes": "Highlight the intense emotional scenes that occur in this module, particularly those tied to cultural values and social norms."
      }},
      "turning_learning_objectives_into_story_points": [
        {{
          "topic": "Topic Name",
          "role_in_story": "Explain what role the topic will play in the story, considering cultural and social influences."
        }}
      ],
      "story_arc": {{
        "premise": "Briefly set the scene, including cultural context such as location, landmarks, and significant social or religious events.",
        "triggering_event": "What event occurs that presents the main financial challenge or opportunity related to this module's learning goal?",
        "rising_action": [
          "Describe a sequence of 3-4 key events that create conflict, tension, or decision points for the characters, incorporating cultural elements like social norms, family obligations, or festival-related pressures."
        ],
        "climax": "What is the most dramatic or pivotal moment in the story where the characters face a significant choice or consequence related to the financial literacy concept, influenced by cultural factors?",
        "falling_action": "How do the events immediately following the climax unfold? How do characters react to the consequences of the choices made, particularly in relation to their cultural or social context?",
        "resolution": "What is the outcome of the story? How is the financial challenge addressed or resolved (or not)? What lessons have the characters learned about the financial concept, and how does this relate to their cultural values?",
        "connecting_thread": "Briefly explain how this module"s story arc connects to the previous one and creates a natural lead-in to the next, considering cultural continuity."
      }}
    }},
    ...
    {{
      "module_number": 5,
      "goal_of_the_module": "Goal of the Module",
      "topics_to_be_covered": [
        {{
          "topic": "Topic Name",
          "description": "Topic Description"
        }}
        ...
        {{
          "topic": "Topic Name",
          "description": "Topic Description"
        }}
      ],
      "featured_characters": {{
        "Character Name 1": "Role",
        ...
        "Character Name n": "Role"
      }},
      "recap_of_previous_module": "Brief summary of the key events and outcomes of the previous module",
      "high_drama_elements": {{
        "conflict": "Describe the main conflict in this module, ensuring it reflects cultural norms and social pressures.",
        "twists": "Detail any unexpected twists that add culturally relevant drama to the story.",
        "emotional_scenes": "Highlight the intense emotional scenes that occur in this module, particularly those tied to cultural values and social norms."
      }},
      "turning_learning_objectives_into_story_points": [
        {{
          "topic": "Topic Name",
          "role_in_story": "Explain what role the topic will play in the story, considering cultural and social influences."
        }}
      ],
      "story_arc": {{
        "premise": "Briefly set the scene, including cultural context such as location, landmarks, and significant social or religious events.",
        "triggering_event": "What event occurs that presents the main financial challenge or opportunity related to this module's learning goal?",
        "rising_action": [
          "Describe a sequence of 3-4 key events that create conflict, tension, or decision points for the characters, incorporating cultural elements like social norms, family obligations, or festival-related pressures."
        ],
        "climax": "What is the most dramatic or pivotal moment in the story where the characters face a significant choice or consequence related to the financial literacy concept, influenced by cultural factors?",
        "falling_action": "How do the events immediately following the climax unfold? How do characters react to the consequences of the choices made, particularly in relation to their cultural or social context?",
        "resolution": "What is the outcome of the story? How is the financial challenge addressed or resolved (or not)? What lessons have the characters learned about the financial concept, and how does this relate to their cultural values?",
        "connecting_thread": "Briefly explain how this module's story arc connects to the previous one and creates a natural lead-in to the next, considering cultural continuity."
      }}
    }}
  ]
}}
```
'''

# Narrative Development Agents
## **Run all the cells from this point to generate a new story with the same curriculum**
Here you will run the code to generate the curriculum in the following steps:
1. Create 5 options of narratives
2. Select the narrative you want to build [INPUT REQUIRED]
3. Draft json with 6 character descriptions
4. Give feedback on character descriptions [INPUT REQUIRED]
5. Create final json with 6 character descriptions
6. Save character descriptions
7. Load saved character descriptions
8. Draft json for the story arc of first 5 modules
9. Draft json for the story arc of last 5 modules
10. Join both json for the final story arcs
11. Save story arc json



In [ ]:
# @title Create 5 Narrative Options

messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": narrative_system},
    {"role": "user", "content": (f"{narrative_options}\n\n"
                                 f"Use the following learning curriculum and information sheet for reference:\n"
                                 f"Curriculum:\n{parsed_curriculum}\n"
                                 f"Source Content:\n{source_content}"
                                )
    }]


response = await azure_openai_client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        temperature=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
)

narrative_approach = response.choices[0].message.content
print(narrative_approach)

In [ ]:
# @title Narrative Selector
select_narrative = input("Select the narrative you want to go ahead with - ")

In [ ]:
#@title Character Profile Template
character_profiles_generation_template = f'''
{{
    "character_number": 1,  # Unique identifier for the character
    "character_role": "string",  # Assign one of the defined character roles, e.g., "Protaganist", "Antagonist"
    "name": "string",  # Reflecting common names in India, e.g., "Rajesh Sharma", "Priya Patel"
    "age": "string",  # Age , e.g., "30", "13" Age should reflect in the relations with other character. for example, a father at a age of 40 can have a daughter of age 20.
    "hometown": "string",  # Place of origin in/near {CITY}, within the state of {STATE}
    "state": "string",  # {STATE} (Keep it {STATE} Unless Specified)
    "ethnicity": "string",  # In relevance to the story and state, considering social and cultural norms.
    "physical_appearance": {{
        "key_traits": "string",  # Key descriptors, e.g., "strong build, confident"
        "features": "string",  # Notable physical features, e.g., "sharp nose, deep-set eyes"
        "personality": "string",  # Personality traits, e.g., "hardworking, resilient"
        "gender": "string",  # Gender identity, e.g., "male", "female"
        "height": "string",  # Height or description, e.g., "average", "tall"
        "build": "string",  # Body type, e.g., "slim", "stocky", "muscular"
        "hair_type": "string",  # Hair texture, e.g., "thick, wavy", "curly"
        "hair_style": "string",  # Hairstyle, e.g., "neatly combed", "braided"
        "hair_color": "string",  # Hair color, e.g., "black", "dark brown"
        "eye_color": "string",  # Eye color, e.g., "dark brown", "black"
        "skin_tone": "string",  # Skin tone, e.g., "wheatish", "dusky", "fair"
        "facial_shape": "string",  # Face shape, e.g., "oval", "round"
        "nose": "string",  # Nose shape, e.g., "aquiline", "broad"
        "lips": "string",  # Lip type, e.g., "thin", "full"
        "eyebrows": "string",  # Eyebrow shape, e.g., "thick", "arched"
        "posture": "string"  # Posture, e.g., "upright", "relaxed"
    }},
    Description of character outfit that suits the story, reflecting the cultural norms of the story location. The clothing should be appropriate to the age and gender of the character. Specify its color, and type.
    **Keep the outfits simple, practical, and minimalistic. Avoid mentioning an extensive range of colors, intricate embroidery, heavy ornamentation, or luxury textiles.**
    Use clear and concise language to describe garments like sarees, dhotis, lungis, salwar kameez, kurtas, and simple blouses, as well as how they are typically worn. Emphasize the functional aspects of the clothing, such as comfort in hot weather, ease of movement for agricultural work, and cultural significance in daily life.
    "outfit": {{
        "outfit_color": "string",  # Dominant clothing color, e.g., "black and white", "pink" [IF there are multiple colors, mention which color is for what in the outfit type. for example, black kurta with white pyjama]
        "outfit_type": "string",  # Type of outfit that suits the story location, reflecting the cultural norms. The attire should be appropriate to the age and gender of the character., e.g., "kurta-dhoti", "salwar-kameez", "shirt and trousers"
        "texture_and_fabric": "string",  # Fabric type, e.g., "cotton", "khadi"
        "footwear": "string"  # Footwear type, e.g., "chappals", "mojari", "formal shoes"
    }},
    Add jewelry and its description if it suits with outfit of the character and the context of the story. Add the specific color and type of jewelry.
    Jewelry should be minimal and limited to essential items, such as simple bangles, small earrings, or basic nose studs, as characters from rural India would not have access to or be able to afford elaborate or decorative accessories.
    "jewelry_details": {{
        "jewelry_type": "string",  # Type of jewelry, e.g.,"small earrings" "bangles", "mangalsutra", "nose ring"
        "jewelry_color": "string"  # Jewelry color, e.g., "gold", "silver", "brass", "iron"
        # Not all characters need to wear jewelry; this field can be left empty or set to "N/A".
    }},
    Accessories should be minimal and limited to practical, essential items that support daily life in rural India. Include only items such as a small backpack, a simple sickle, a basic stick, or other functional tools relevant to the local lifestyle. Avoid any decorative, ornamental, or luxury accessories.
    "accessory_details": {{
        "accessory_type": "string",  # Accessory type, e.g., "gamcha", "pagdi", "dupatta", "backpack", "purse"
        "accessory_color": "string"  # Accessory color, e.g., "red", "green" simple block colors
        # Not all characters need accessories; this field can also be left empty or set to "N/A".
    }},
    "occupation": "string",  # Profession or livelihood, e.g., "farmer", "teacher", "artisan"
    "financial_personality": "string",  # Key traits and how they might influence financial decisions, e.g., "Traditional and cautious, often wary of modern banking"
    "cultural_background": "string",  # Cultural heritage. Details about the character"s region, language, religion, and traditional practices., e.g., "Comes from a conservative Brahmin family where traditional values are emphasized, and she"s accustomed to community life with rich customs and festivals"
    "brief_backstory": "string",  # Highlighting their relationship to financial matters, including any relevant cultural practices., e.g., "Widowed at a young age, Priya lives with her son and faces societal pressures and financial instability."
    "relationship_to_other_characters": "string",  # Description of relationships with other characters, considering social and cultural norms., e.g., "younger sibling of the protagonist", "mentor of the main character"
    "potential_for_growth_change": "string"  # How might this character evolve through the story, particularly in relation to cultural and social influences, e.g., "Has great potential to transition from a traditional homemaker to a financially empowered individual and community leader by embracing modern finance principles and teaching others."
}}

'''

In [ ]:
# @title Draft Character Profiles


from typing import List, Optional
from pydantic import BaseModel, Field

class PhysicalAppearance(BaseModel):
    key_traits: str = Field(description="Keywords describing the character's main physical traits.", examples=["young", "tall", "athletic", "fragile"])
    features: str = Field(description="Distinctive physical features of the character.", examples=["sharp jawline", "dimples", "freckles"])
    personality: str = Field(description="The character's dominant personality traits.", examples=["introverted", "fiery", "calm"])
    gender: str = Field(description="The gender identity of the character.", examples=["male", "female", "non-binary"])
    height: str = Field(description="The character's height.", examples=["tall", "short", "5'9\""])
    build: str = Field(description="The character's body build.", examples=["slim", "muscular", "average"])
    hair_type: str = Field(description="The texture of the character's hair.", examples=["curly", "straight", "wavy"])
    hair_style: str = Field(description="How the character's hair is styled.", examples=["Tied", "ponytail", "loose"])
    hair_color: str = Field(description="The color of the character's hair.", examples=["blonde", "black", "auburn"])
    eye_color: str = Field(description="The color of the character's eyes.", examples=["blue", "brown", "hazel"])
    skin_tone: str = Field(description="The character's skin complexion.", examples=["fair", "olive", "dark"])
    facial_shape: str = Field(description="The shape of the character's face.", examples=["oval", "square", "heart-shaped"])
    nose: str = Field(description="The shape of the character's nose.", examples=["button", "aquiline", "broad"])
    lips: str = Field(description="The shape and size of the character's lips.", examples=["thin", "full", "pouty"])
    eyebrows: str = Field(description="The shape and thickness of the character's eyebrows.", examples=["arched", "bushy", "thin"])
    posture: str = Field(description="The character's typical posture.", examples=["upright", "slouched", "rigid"])

class Outfit(BaseModel):
    outfit_color: str = Field(description="The primary color of the character's outfit.", examples=["red", "blue"])
    outfit_type: str = Field(description="The style or type of the character's clothing.", examples=["formal suit", "saree", "kurta-dhoti"])
    texture_and_fabric: str = Field(description="The material and texture of the outfit.", examples=["cotton", "velvet", "silk"])
    footwear: str = Field(description="The type of footwear the character is wearing.", examples=["bare-foot", "sandals", "shoes"])

class Jewelry(BaseModel):
    jewelry_type: Optional[str] = Field(default=None, description="The type of jewelry the character is wearing.", examples=["no jewelery", "bracelet", "ring"])
    jewelry_color: Optional[str] = Field(default=None, description="The color or material of the jewelry.", examples=["N/A", "gold", "silver", "brass"])

class Accessory(BaseModel):
    accessory_type: Optional[str] = Field(default=None, description="The type of accessory the character has.", examples=["turban", "paghadi", "belt"])
    accessory_color: Optional[str] = Field(default=None, description="The color of the accessory.", examples=["brown", "black"])

class CharacterProfile(BaseModel):
    character_number: int = Field(description="A unique identifier for the character.")
    character_role: str = Field(description="The role of the character in the story.")
    name: str = Field(description="The name of the character.")
    age: str = Field(description="The age of the character.")
    hometown: str = Field(description="The character's hometown.")
    state: str = Field(description="The state where the character is from.")
    ethnicity: str = Field(description="The character's ethnic background.")

    # Nested Models
    physical_appearance: PhysicalAppearance = Field(description="Detailed physical appearance of the character.")
    outfit: Outfit = Field(description="Details about the character's outfit.")
    jewelry_details: Jewelry = Field(description="Details about the character's jewelry.")
    accessory_details: Accessory = Field(description="Details about the character's accessories.")

    # Additional Attributes
    occupation: str = Field(description="The character's profession or job.", examples=["businessman", "doctor", "engineer"])
    financial_personality: str = Field(description="The character's attitude and behavior towards money.", examples=["frugal", "extravagant", "indifferent"])
    cultural_background: str = Field(description="The cultural context of the character.", examples=["Indian", "Mexican", "European"])
    brief_backstory: str = Field(description="A brief summary of the character's history or key life events.")
    relationship_to_other_characters: str = Field(description="How the character relates to other characters in the story.", examples=["friend", "rival", "mentor"])
    potential_for_growth_change: str = Field(description="Areas where the character can develop or change throughout the story.", examples=["overcomes fear of failure", "learns to trust others"])


class StoryModel(BaseModel):
    character_profiles: List[CharacterProfile]


messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": narrative_system},
    {"role": "user", "content": (f"{narrative_options}\n\n"
                                 f"Use the following learning curriculum and information sheet for reference:\n"
                                 f"Curriculum:\n{parsed_curriculum}\n"
                                 f"Source Content:\n{source_content}"
                                )
    },
    {"role": "assistant", "content": f"{narrative_approach}"},
    {"role": "user", "content": (
        f"Out of these, lets move ahead with Narrative Approach {str(select_narrative)}\n\n"
        f"{narrative_options}\n\n"
        f"First, lets generate the Character Profiles"
        f"Use the following character profile tamplate generation template for reference:\n"
        f"{character_profiles_generation_template}"
        f"Physical traits and outfits: \n {looks_and_outfits}"
)}]


response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=1.0,
    response_format=StoryModel
)

storyarc_character_profile_gen = response.choices[0].message.content
storyarc_character_profile_gen = json.loads(storyarc_character_profile_gen)
for profile in storyarc_character_profile_gen['character_profiles']:
    print(f"Character {profile['character_number']}:")
    for key, value in profile.items():
        print(f"  {key}: {value}")
    print("\n" + "-" * 50)

In [ ]:
#@title Feedback on Character Profile
feedback_character_profile = input("Give feedback on character profiles : ")


In [ ]:
#@title Final Character Profile
messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": narrative_system},
    {"role": "user", "content": (f"{narrative_options}\n\n"
                                 f"Use the following learning curriculum and information sheet for reference:\n"
                                 f"Curriculum:\n{parsed_curriculum}\n"
                                 f"Source Content:\n{source_content}"
                                )
    },
    {"role": "assistant", "content": f"{narrative_approach}"},
    {"role": "user", "content": (
        f"Out of these, lets move ahead with Narrative Approach {str(select_narrative)}\n\n"
        f"{narrative_options}\n\n"
        "First, lets generate the Character Profiles")},
    {"role": "assistant", "content": f"{storyarc_character_profile_gen}",},
    {"role": "user", "content": f"User Feedback on characters profiles: {feedback_character_profile}"}

]


response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=1.0,
    response_format=StoryModel
)

final_character_profile_gen = response.choices[0].message.content
final_character_profile_gen = json.loads(final_character_profile_gen)
for profile in final_character_profile_gen['character_profiles']:
    print(f"Character {profile['character_number']}:")
    for key, value in profile.items():
        print(f"  {key}: {value}")
    print("\n" + "-" * 50)

In [ ]:
# @title Saving Characters Profile

save_file_to_folder(folder_path, f"character_profiles.json", json.dumps(final_character_profile_gen))

In [ ]:
# @title Load characters from prev saved files

character_profiles = load_save_files(RUN_TIME, f"character_profiles.json")

In [ ]:
# @title Story Arc Agent
class OverarchingNarrative(BaseModel):
    main_plot: str
    setting: str
    context_of_story: str
    central_conflict: str


class Topic(BaseModel):
    topic: str
    description: str


class HighDramaElements(BaseModel):
    conflict: str
    twists: str
    emotional_scenes: str


class LearningObjectiveStoryPoint(BaseModel):
    topic: str
    role_in_story: str


class StoryArc(BaseModel):
    premise: str
    triggering_event: str
    rising_action: List[str]
    climax: str
    falling_action: str
    resolution: str
    connecting_thread: str


class ModuleStoryArc(BaseModel):
    module_number: int
    goal_of_the_module: str
    topics_to_be_covered: List[Topic]
    featured_characters: Optional[Dict[str, str]]
    recap_of_previous_module: str
    high_drama_elements: HighDramaElements
    turning_learning_objectives_into_story_points: List[LearningObjectiveStoryPoint]
    story_arc: StoryArc


class ModuleStoryArcs(BaseModel):
    module_story_arcs: List[ModuleStoryArc]

messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": narrative_system},
    {"role": "user", "content": (f"{narrative_options}\n\n"
                                 f"Use the following learning curriculum and information sheet for reference:\n"
                                 f"Curriculum:\n{parsed_curriculum}\n"
                                 f"Source Content:\n{source_content}"
                                )},
    {"role": "assistant", "content": f"{narrative_approach}"},
    {"role": "user", "content": (
        f"Out of these, lets move ahead with Narrative Approach {str(select_narrative)}\n\n"
        f"{narrative_options}\n\n"
        "First, lets generate the Character Profiles")},
    {"role": "assistant", "content": f"{character_profiles}"},
    {"role": "user", "content": "Now, let's generate the *first 5*  Module Arcs."}]

response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=1.0,
    response_format=ModuleStoryArcs
)

five_modules = response.choices[0].message.content

In [ ]:
module_arcs = json.loads(five_modules)
for module in module_arcs['module_story_arcs']:
    print(f"Module {module['module_number']}:")
    print(f"Goal: {module['goal_of_the_module']}")
    print("\nTopics to be Covered:")
    for topic in module["topics_to_be_covered"]:
        print(f"- {topic['topic']}: {topic['description']}")
    print("\nFeatured Characters:")
    for character, role in module["featured_characters"].items():
        print(f"- {character}: {role}")
    print("\nHigh Drama Elements:")
    for element, detail in module["high_drama_elements"].items():
        print(f"- {element.capitalize()}: {detail}")
    print("\nStory Arc:")
    for arc_part, detail in module["story_arc"].items():
        if isinstance(detail, list):
            print(f"- {arc_part.capitalize()}:")
            for item in detail:
                print(f"  - {item}")
        else:
            print(f"- {arc_part.capitalize()}: {detail}")
    print("\n" + "="*50 + "\n")


In [ ]:
# @title Join the arcs in one single module
# Combine the module_story_arcs from both parts
five_modules = json.loads(five_modules)

narrative = {
    "module_story_arcs": five_modules['module_story_arcs']
}

# Sort the combined module_story_arcs by module_number
narrative["module_story_arcs"].sort(key=lambda x: int(x["module_number"]))

# Print the combined JSON
print(narrative)

In [ ]:
# @title Save Narrative
save_file_to_folder(folder_path, f"/narrative.json", json.dumps(narrative))

# Story Drafting Prompts
YOU CAN RUN THIS BLOCK WITHOUT INPUTS
- Here are all the prompts used while drafting the story. You can edit the prompts, but do not change the name of the prompts.

In [ ]:
# @title Drafting Agent System Prompt

draft_system_1 = f"""
You are a drafting agent specializing in writing clear, engaging, and educational stories about financial literacy. You write compelling and informative stories for communities in rural and semi-urban India. Your task is to draft a 5-module story-based financial literacy curriculum. Each module in this curriculum will be 1000 words.

### Objective:
- Draft a 1000-word educational story for a given module that teaches financial literacy concepts in an engaging and relatable manner based on the source materials provided.
- The story should be a single continuous narrative without subsections or headings.
- Incorporate culturally relevant elements to make the narrative resonate with the target audience.

### Tone and Style:
The vocabulary you use will be simple, and you will avoid phrases, words or other grammatical instruments that are difficult to translate to indic languages.
Use third person narrative and first person dialogue.
The tone of your writing should be narrative fiction.
Keep your sentences short.
Ensure that the narrative is filled with high drama and soap opera-like elements. Include dramatic conflicts, unexpected twists, and intense emotional scenes to keep the audience engaged.

### Audience:
- The target audience is low-income Indians in rural and semi-urban areas with limited experience with formal financial systems.Their English proficiency may be very weak, so use very simple and easy-to-understand English.
{target_audience}

### Source Materials:
You have access to the following materials:
**Source File with Domain Information**:
    - This file provides comprehensive information about financial concepts and practices relevant to the curriculum.
**Financial Literacy Curriculum**
This file details all the 5 modules of the curriculum and the topics covered in them.
** Character Profiles**:
    - These provide the main plot, key characters, their cultural backgrounds, and their roles in the story. Pay attention to details such as clothing, language, religious practices, and social norms.
**Overarching Narrative and Module Story Arc** :
    - These include the goal of the module, topics to be covered, featured characters, turning learning objectives and topics into story points and narrative devices, incorporation of learning topics into the story arc, and the story arc itself.
    - Ensure the integration of cultural elements such as location, landmarks, scenery, festivals, and traditional practices within the story arc.


### Key Points:
- Develop a 1000-word story for each module that is educational, engaging, and relatable for low-income Indians.
- Create a compelling narrative that effectively teaches financial literacy concepts through well-developed characters and storytelling.
- Ensure that only the financial topics relevant to the current module are included in each story.
- Use simple, clear language appropriate for readers with weak English proficiency.
- Integrate cultural elements seamlessly into the narrative, including regional settings, social norms, values, religious practices, festivals, and traditional customs.
- Ensure that the characters' cultural backgrounds are reflected in their actions, dialogues, and interactions.

Your role is to synthesize these materials into a coherent, engaging, and educational narrative that achieves the learning objectives while maintaining reader interest throughout the 1000-word story.
"""

In [ ]:
# @title Draft Generation Prompt

draft_generation_1 = '''
Draft a 1000-word educational story for a given module that teaches financial literacy concepts in an engaging and relatable manner. Use the material provided to you for reference.

### Writing Style Guidelines:
1. **Expository and Explicit**:
   - Clearly explain financial literacy concepts and processes. Ensure the educational content is explicit and easy to understand, even if it becomes somewhat pedantic.
   - The 1000-word story must include clear explanations of the financial topics relevant to the current module.

2. **Straightforward and Simple English**:
   - Avoid using excessive adjectives and sensory descriptions. The writing should be straightforward, concise, and to the point.
- Avoid using phrases, words or other grammatical structures that are difficult to translate into Indic languages.
   - Use very simple and easy-to-understand English to ensure accessibility for readers with weak English proficiency.
   - Focus on actions, dialogues, and educational content without unnecessary descriptive elements.

3. **Strong Narrative**:
   - Develop a compelling narrative that progresses through a sequence of events. Ensure the protagonist's journey is engaging and relatable.
   - Maintain a logical flow, with each module building on the previous one.
   - The story should be a single continuous prose, without any subheadings or sections.

4. **Interest and Drama**:
   - The story should have interesting elements to keep the reader engaged.
   - Include culturally relevant drama and tension, such as conflicts arising from social norms, family obligations, or cultural events.
   - Ensure the dramatic elements contribute to the educational goals and overall storyline.

5. **Third-Person Perspective with First-Person Dialogues**:
   - Narrate the story in the third person to maintain an objective tone.
   - Use first-person dialogues to make the conversations engaging and personal.

6. **Accuracy and Consistency of Financial Literacy Content**:
   - Financial literacy concepts should be accurate, clear, and well-integrated into the narrative.
   - Only include the financial topics relevant to the current module in the narrative.

### Character and Story Integration:
  - Use the character profiles, module outlines, and information from the source file and curriculum development output to develop the story.
  - Ensure that characters' actions, dialogues, and cultural practices align with the educational goals of each module.
  - Seamlessly integrate financial literacy concepts into the narrative without breaking the flow.
  - Incorporate cultural elements like ecological elements, location, landmarks, scenery, festivals, traditional food, and social customs to enrich the narrative and make it relatable.

### Integration of Learning Topics into the Story Arc:
  - Incorporate only the financial topics relevant to the current module into the narrative.
  - Use the cultural context and characters' backgrounds to naturally introduce financial concepts within the story.

### Incorporating Cultural Elements into the story
Make the incorporation of cultural elements into the story feel natural, do not add unnecessary descriptions of cultural markers.

### Using Previous Modules for Continuity:
  - Reference previous modules to maintain continuity and coherence in the protagonist's journey, including their evolving understanding of financial literacy concepts and how these are influenced by their cultural context.

### Task: Draft the Complete Story for the Module:
  - Draft the 1000-word story for the given module, ensuring alignment with the overall narrative and educational objectives.
  - The story should be a single 1000-word story with good flow and development in the plot.
  - Prioritize clarity, engagement, cultural relevance, and educational value to effectively teach financial literacy concepts through storytelling.
  - Do not make subsections inside the story; write a single complete story.
'''

### Cultured Module-Specific Instruction

module_1_specific_instruction = """
As the current module is the first module, give an introduction of the protagonist and their context, current situation, history, and immediate relationships. Use the first paragraph of the story to provide this introduction, including culturally relevant details such as the protagonist's region, social norms, clothing, and the local environment. Introduce any significant cultural practices, festivals, or community events that may influence the protagonist’s financial decisions or worldview.
"""

# Story Drafting Agent
Here you will run the code to draft the full curriculum in the following steps.
1. Run Code

In [ ]:
!pip install -q tqdm
from tqdm import tqdm

previous_modules = ''

for module_number in tqdm(range(1,6)):

  narrtive_module_info = narrative['module_story_arcs'][module_number-1]

  if module_number == 1:
    content_req = f"""Use the following information to draft a 1000-word module.\n
          Module Number: {module_number}\n
          Overarching Narrative and Character Information:\n{str(final_character_profile_gen)}\n
          Narrative information and stories for all Modules:\n{str(narrative)}\n\n
          Source File with Domain Information:\n{source_content}\n
          -------\n
          Narrative Information for current module: {narrtive_module_info}"""
  else:
    content_req = f"""Continue the story from previous modules.\n
              Previous Module Stories:\n{previous_modules}\n
              Module Number: {module_number}\n
              Overarching Narrative and Character Information:\n{str(final_character_profile_gen)}\n
              Narrative information and stories for all Modules:\n{str(narrative)}\n\n
              Source File with Domain Information:\n{source_content}\n
              -------\n
              Narrative Information for current module: {narrtive_module_info}"""

  response = await azure_openai_client.beta.chat.completions.parse(
      messages=[
      {
        "role": "system",
        "content": draft_system_1
      },
      {
        "role": "user",
        "content": content_req
      }
    ],
    model="gpt-4o",
    temperature=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  previous_modules = f"\n\n------------\n\n".join([previous_modules, response.choices[0].message.content])

## save narrative as a text file
save_path = save_file_to_folder(folder_path, f"stories.txt", previous_modules)

# Scene Splitting Agent

## Scene Splitting Prompts

YOU CAN RUN THIS BLOCK WITHOUT INPUT

In [ ]:
# @title System Prompt
Scene_split_system = """ You are an expert illustrator of educational stories. You will analyse the given financial literacy curriculum through the lens of developing appropriate images for each of the modules within it. All the images you create will be culturally accurate and the characters will have a visual consistency throughout the story, just like comic book characters.

The audience for this curriculum are communities from rural and semi-urban India.

Through the story or through your imagination you create and pay attention to the following elements:

Key Characters:
  - Physical attributes (age, appearance, style)
  - Personality traits
  - Cultural background (clothing, accessories)
  - Consistent visual elements that appear in every scene, making the character recognisable
  - Current emotions
  - Dynamic relationships with other characters

Key Locations
  - Consistent visual elements that make the location recognisable
  - Cultural appropriateness
  - Geographical consistency

Key Scenes:
  - The description of the setting
  - Time of day
  - Character emotions / interactions
  - Key objects or symbols related or financial concepts
  - Cultural elements that would make images relatable for your audience

Visual Enhancers for every identified scene:
  - Perspective/Camera Angle
  - Background details
  - Cultural artifacts or environmental elements
"""

In [ ]:
# @title Generation Prompt

scene_split_generation= """ Respond with at least 4000 tokens. Split each of the given modules into scenes that will be relevant for illustrations. Use the character descriptions generated earlier to inform your outputs and make sure to generate detailed responses for every module, every character, and every scene.
respond with the scenes for the current module only.
Scene Breakdown Process Template
Initial Module Analysis

For each module, consider:

Overall story arc and learning objectives
Natural breaking points for visual storytelling
Dramatic tension and pacing
Logical progression of events


Scene Count Decision Criteria:

Must be between 6 scenes
Consider:

Complexity of material
Number of key moments
Required character interactions
Location changes
Time progression



Scene Development Process
For each scene:
1. Character Selection

Identify required characters based on:

Scene objectives
Story progression
Necessary interactions


For each character, detail:

Current emotional state (tied to scene events)
Outfit choices (considering):

Character's role
Scene location
Time period
Weather conditions
Cultural context


Cultural markers (minimum 2):

Physical attributes
Behavioral traits
Cultural symbols

2. Action Sequence Development
describe the action such a way that it can used for a still scene visual. The description will have description of how the character will look like in that particular pose in the scene.
for example
"Rohan's arm is extended, his palm open and fingers slightly curled, frozen in the moment before contact. His posture remains steady, weight evenly distributed, with his gaze fixed ahead, unwavering. The subtle creases in his sleeve hint at movement just halted, while the tension in his fingers suggests anticipation. The warm glow of the village lantern casts a soft shadow along his forearm, emphasizing the openness of his gesture. Time seems suspended, the handshake waiting to be completed."

Main action description
Which characters are involved
Physical positioning in scene with respect to the viewer's point of view


Detail character poses:
pose according during the action in the scene

action in the scene:
- Shaking Hands (e.g., Rohan): If Rohan is shaking hands with another character, describe only the physical action of Rohan's hand and arm extending, palm open, ready for the handshake. Do not mention the other person involved in the handshake. For example: "Rohan's arm is extended, palm open, fingers slightly curled in anticipation of the handshake."
- Supporting from Behind (e.g., Sita): If Sita is standing behind Gita with her arm on Gita's shoulder for support, describe Sita's arm as extended and resting lightly on a surface (implied shoulder), hand gently curved. Do not mention Gita. For example: "Sita's arm is gently draped, hand resting lightly with fingers slightly curled." Similarly, describe Gita as slightly leaning forward, posture relaxed but not fully slumped, without specifying the source of support. For example: "Gita's posture is relaxed, with a slight forward lean."
- Hugging from Behind (e.g., Vikram): If Vikram is hugging Anika from behind, describe the action as Vikram standing close behind someone, with his arms encircling a space (implied waist area), hands gently clasped. Do not mention Anika. For example: "Vikram's arms are wrapped around, hands gently clasped together."


3. Setting Construction

Location details:
Primary location is a specific location for e.g., bank, marketplace, [protaganist name]'s house
Keep the primary location as general as possible which can be repetively used in other scenes of the story for example the protaganist's house is a primary location and will be used in other scenes.
Secondary location is a specific spot inside the primary location for e.g., bank manager's office, vegeatble stall, [protaganist name]'s living room/bedroom,etc.)
Keep the secondary location specific to the scene for example kitchen inside the protaganist's house is a secondary location within primary.
Specific place
Broader geographical context
Cultural significance


Environmental factors:

Time of day
Weather conditions
Seasonal elements


Key location elements:

Physical landmarks
Important objects
Atmospheric details

4. Visual Elements

Technical considerations:

Camera angle selection
Framing decisions
Focal points


Cultural elements:

Relevant artifacts
Symbolic objects
Environmental markers


Financial/Economic indicators:

Visible wealth markers
Economic status symbols
Business/financial elements



5. Mood and Description

Overall mood:

Emotional atmosphere
Tension level
Energy/pace


Scene description:

Opening setup
Key moments
Closing state



JSON Structure Rules

Maintain consistent formatting:

Proper nesting
Complete object closure
Array formatting


Field requirements:

All fields must be filled
No null values
Arrays must have at least one element


Validation checks:

Scene numbers match declared count
Character references consistent
All required fields present


Generate your output in the following json format:
```json

{
  "modules": [  # A collection of modules, each representing a narrative segment
    {
      "module_number": "1",  # Unique identifier for the module
      "number_of_scenes": "5",  # Number of scenes in the module, keep in range of 4 to 7
      "scenes": [  # List of scenes within the module
        {
          "scene_number": "1",  # Unique identifier for the scene
          "title": "string",  # Title or name of the scene
          "characters_present": [  # Characters involved in the scene. Only use the characters from the character description
            {
              "character_number": "1",  # Unique identifier for the character
              "name": "string",  # Name of the character
              "current_emotion": "string",  # Current emotional state of the character
              "cultural_markers": ["string", "string"],  # Cultural or personal markers of the character
              "specific_actions":  # Specific actions happening in the scene by each character
              {
                "action_description": [
                "Rohan's arm extends forward, his palm open, fingers slightly curled in anticipation of contact. A brief pause lingers in the air before the inevitable clasp."
                ],  # Description of the action in a scene-like manner
                "positioning": [
                "Standing near the entrance of the village hall, facing Aryan"
                ],  # Positioning of characters relative to the scene
                "pose": [
                "Right arm extended",
                "Slightly leaning forward"
                ],  # Pose of the character, describing their physical stance
                "action": [
                "Extending hand for a handshake",
                "Maintaining a firm posture, gaze locked on Aryan"
                ]  # Action performed by the character, framed as part of a scene
                }
            },
            ...
            {
              "character_number": "n",  # Unique identifier for the character
              "name": "string",  # Name of the character
              "current_emotion": "string",  # Current emotional state of the character
              "cultural_markers": ["string", "string"],  # Cultural or personal markers of the character
              "specific_actions":  # Specific actions happening in the scene by each character
              {
                "action_description": [
                "Rohan's arm extends forward, his palm open, fingers slightly curled in anticipation of contact. A brief pause lingers in the air before the inevitable clasp."
                ],  # Description of the action in a scene-like manner
                "positioning": [
                "Standing near the entrance of the village hall, facing Aryan"
                ],  # Positioning of characters relative to the scene
                "pose": [
                "Right arm extended",
                "Slightly leaning forward"
                ],  # Pose of the character, describing their physical stance
                "action": [
                "Extending hand for a handshake",
                "Maintaining a firm posture, gaze locked on Aryan"
                ]  # Action performed by the character, framed as part of a scene
                }
            }
          ],

          "setting": {  # Description of the scene's setting
            "primary_location": "string",  # Primary location of the scene (can be used in other scenes(keep the same when used in other scenes)) (e.g., bank, marketplace, [protaganist name]'s house)
            "secondary_location": "string",  # Secondary location within the primary location (e.g., bank manager's office, vegeatble stall, [protaganist name]'s living room/bedroom,etc.)
            "geographical_context": "string",  # Geographical context of the scene
            "geographical_location": "string",  # Specific geographical location, e.g., coordinates or region
            "time_of_day": "string",  # Time of day for the scene
            "weather": "string",  # Weather during the scene
            "key_location_elements": ["string", "string"]  # Key elements that define the location
          },
          "visual_elements": {  # Visual elements enhancing the scene
            "camera_angle": "string",  # Camera angle used to capture the scene
            "cultural_artifacts": ["string", "string"],  # Cultural artifacts present in the scene
            "financial_symbols": ["string", "string"]  # Use ONLY Indian Rupees symbol for financial symbols in the scene. Use ONLY INR currency notes or coins when cash or money is described in the story. Prefer using currency notes.
          },
          "mood": "string",  # Overall mood of the scene
          "description": "string"  # Detailed description of the scene
        },
        ...
        {
          "scene_number": "1",  # Unique identifier for the scene
          "title": "string",  # Title or name of the scene
          "characters_present": [    # Characters involved in the scene. Only use the characters from the character description
            {
              "character_number": "1",  # Unique identifier for the character
              "name": "string",  # Name of the character
              "current_emotion": "string",  # Current emotional state of the character
              "cultural_markers": ["string", "string"],  # Cultural or personal markers of the character
              "specific_actions":  # Specific actions happening in the scene by each character
              {
                "action_description": [
                "Rohan's arm extends forward, his palm open, fingers slightly curled in anticipation of contact. A brief pause lingers in the air before the inevitable clasp."
                ],  # Description of the action in a scene-like manner
                "positioning": [
                "Standing near the entrance of the village hall, facing Aryan"
                ],  # Positioning of characters relative to the scene
                "pose": [
                "Right arm extended",
                "Slightly leaning forward"
                ],  # Pose of the character, describing their physical stance
                "action": [
                "Extending hand for a handshake",
                "Maintaining a firm posture, gaze locked on Aryan"
                ]  # Action performed by the character, framed as part of a scene
            }
            },
            ...
            {
              "character_number": "n",  # Unique identifier for the character
              "name": "string",  # Name of the character
              "current_emotion": "string",  # Current emotional state of the character
              "cultural_markers": ["string", "string"],  # Cultural or personal markers of the character
              "specific_actions":  # Specific actions happening in the scene by each character
              {
                "action_description": [
                "Rohan's arm extends forward, his palm open, fingers slightly curled in anticipation of contact. A brief pause lingers in the air before the inevitable clasp."
                ],  # Description of the action in a scene-like manner
                "positioning": [
                "Standing near the entrance of the village hall, facing Aryan"
                ],  # Positioning of characters relative to the scene
                "pose": [
                "Right arm extended",
                "Slightly leaning forward"
                ],  # Pose of the character, describing their physical stance
                "action": [
                "Extending hand for a handshake",
                "Maintaining a firm posture, gaze locked on Aryan"
                ]  # Action performed by the character, framed as part of a scene
                }
            }
          ],

          "setting": {  # Description of the scene's setting
            "primary_location": "string",  # Primary location of the scene (can be used in other scenes(keep the same when used in other scenes)) (e.g., bank, marketplace, [protaganist name]'s house)
            "secondary_location": "string",  # Secondary location within the primary location (e.g., bank manager's office, vegeatble stall, [protaganist name]'s living room/bedroom,etc.)
            "geographical_context": "string",  # Geographical context of the scene
            "geographical_location": "string",  # Specific geographical location, e.g., coordinates or region
            "time_of_day": "string",  # Time of day for the scene
            "weather": "string",  # Weather during the scene
            "key_location_elements": ["string", "string"]  # Key elements that define the location
          },
          "visual_elements": {  # Visual elements enhancing the scene
            "camera_angle": "string",  # Camera angle used to capture the scene
            "cultural_artifacts": ["string", "string"],  # Cultural artifacts present in the scene
            "financial_symbols": ["string", "string"]  # Use ONLY Indian Rupees symbol for financial symbols in the scene. Use ONLY INR currency notes or coins when cash or money is described in the story. Prefer using currency notes.
          },
          "mood": "string",  # Overall mood of the scene
          "description": "string"  # Detailed description of the scene
        },
      ]
    }
  ]
}

```

Implementation Steps:

Start with module-level planning
Develop scene sequence
Detail each scene following the above structure
Review for consistency and completeness
Validate JSON format
Check character continuity across scenes

Quality Checklist:

 All scenes contribute to overall narrative
 Character actions are meaningful and purposeful
 Settings are well-defined and consistent
 Visual elements support scene objectives
 Emotional progression is clear
 Technical requirements are met

let's do this step by step
"""


## Scene Split Agent

1. Load Modules
2. Load Character Profiles
3. Split Module into scenes



In [ ]:
modules = load_save_files(RUN_TIME, f"stories.txt")
characters = load_save_files(RUN_TIME, f"character_profiles.json")

# clean the modules for scene generations
scene_modules = modules.split("\n\n------------\n\n")
scene_modules = scene_modules[1:]

In [ ]:
class SpecificAction(BaseModel):
    action_description: str  # Detailed action description, e.g., "XYZ is handing a book to ABC"
    positioning: str  # Positioning of each character, e.g., "XYZ is standing to the left of the screen"
    pose: str  # Pose character, e.g., "XYZ is sitting, ABC is leaning on the table"
    action: str  # Action of character, e.g., "XYZ is handing over a book, ABC is reaching out"

class Setting(BaseModel):
    primary_location: str  # Primary location, e.g., "marketplace in a small Indian village", "Protagonist's house"
    secondary_location: str  # Secondary location inside the primary location, e.g., "a tea stall in the marketplace", "the living room in the protagonist's house"
    geographical_context: str  # Background details, e.g., "surrounded by rice fields"
    geographical_location: str # Location of the particular state, e.g., District or State
    time_of_day: str  # Specific time of day, e.g., "early morning"
    weather: str  # Description of the weather, e.g., "light drizzle"
    key_location_elements: List[str]  # Key items in the location, e.g., "wooden stalls, cobblestone path"

class VisualElements(BaseModel):
    camera_angle: str  # Perspective, e.g., "over-the-shoulder view"
    cultural_artifacts: List[str]  # Specific cultural items visible, e.g., "brass diya lamps"
    financial_symbols: List[str]  # Financial symbols or indicators, e.g., "currency notes, coins"

class Character(BaseModel):
    character_number: str  # Unique identifier for the character
    name: str  # Character's name
    current_emotion: str  # Specific emotions, e.g., "happy and hopeful," "a sad smile"
    cultural_markers: List[str]  # Cultural elements specific to the character
    specific_actions: SpecificAction  # List of specific actions and positions

class Scene(BaseModel):
    scene_number: str  # Identifier for the scene
    title: str  # Title of the scene
    characters_present: List[Character]  # List of all the characters_numbers in the scene
    setting: Setting  # Scene setting details
    visual_elements: VisualElements  # Description of visual elements
    mood: str # Narrative mood text (set max_length as required), e.g., "A mix of relief and quiet hope"
    description: str  # Summarizing the scene, e.g., "The characters resolve their differences at dawn."

class Module(BaseModel):
    module_number: str  # Identifier for the module
    number_of_scenes: str # Specify the number of scene per module
    scenes: List[Scene] # Scenes in each module

class AllModules(BaseModel):
    modules: List[Module]  # Collection of all modules

messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": Scene_split_system},
    {"role": "user", "content": (f"{scene_split_generation}\n\n"
                                    f"Use full curriculum and stories in coherence to generate scenes"
                                     f"Here is the full curriculum : {scene_modules}\n\n"
                                     f"Here is the original character description : {characters}"
                                    )},
]

response = await azure_openai_client.beta.chat.completions.parse(
    model=model,
    max_tokens=10000,
    messages=messages,
    temperature=1.0,
    response_format=AllModules
)

scene_splitting_response = response.choices[0].message.content
scene_splitting_response = json.loads(scene_splitting_response)

print(scene_splitting_response)

In [ ]:
for module in scene_splitting_response['modules']:
    for scene in module['scenes']:
        print(f"Scene {scene['scene_number']}: {scene['title']}")
        print(f"\tprimary location: {scene['setting']['primary_location']}")
        print(f"\tsecondary location: {scene['setting']['secondary_location']}")

In [ ]:
from tqdm import tqdm
previous_scenes = {"modules": []}

for module_number in tqdm(range(1,6)):
    if module_number == 1:
      content = f"""{scene_split_generation}\n\n
                Only respond with the scenes of the current module
                Current module: {scene_modules[module_number-1]}\n\n
                Use full curriculum and stories in coherence to generate scenes
                Here is the full curriculum : {modules}\n\n
                Here is the original character description : {characters}"""
    else:
      content = f"""{scene_split_generation}\n\n
                Only respond with the scenes of the current module
                Use the scenes from previous modules as a reference
                Previous modules: {previous_scenes}\n\n
                Current module: {scene_modules[module_number-1]}\n\n
                avoid repetition in the scenes
                avoid reference to previous modules in your responses
                for example:
                  avoid 'location': 'Same as previous'
                  try and explicitely mention values in the response such as 'location': 'Lakhsmi's house'
                set module number as: {module_number}\n\n
                Use full curriculum and stories in coherence to generate scenes
                Here is the full curriculum : {modules}\n\n
                Here is the original character description : {characters}"""

    messages: list[ChatCompletionMessageParam] = [
    {"role": "system", "content": Scene_split_system},
    {"role": "user", "content": content},
    ]

    response = await azure_openai_client.beta.chat.completions.parse(
        model=model,
        max_tokens=4000,
        messages=messages,
        temperature=1.0,
        response_format=Module
    )

    scene_splitting_response = response.choices[0].message.content
    scene_splitting_response = json.loads(scene_splitting_response)

    previous_scenes["modules"].append(scene_splitting_response)

In [ ]:
#@title Save Scene Splitting
save_path = save_file_to_folder(folder_path, f"scene_splitting.json", json.dumps(previous_scenes))

In [ ]:
# pretty print the scene splitting response

for module in previous_scenes['modules']:
    print(f"{'-'*50}")
    print(f"\t\tModule {module['module_number']}\n{'-' * 50}\n\n")
    print(f"number of scenes: {module['number_of_scenes']}")

    print("\nScenes:\n")
    for scene in module['scenes']:
        print(f"- Scene {scene['scene_number']}\n")
        print(f"Title: {scene['title']}")

        print("\n- Characters:")
        for character in scene['characters_present']:
            print(f"\nCharacter {character['character_number']}: {character['name']}")
            print(f"Current Emotion: {character['current_emotion']}")
            print(f"Cultural Markers: {', '.join(character['cultural_markers'])}")
            print(f"- Specific Action:")
            print("\n")
            print(f" Action description: {character['specific_actions']['action_description']}")
            print(f" Positioning: {character['specific_actions']['positioning']}")
            print(f" Pose: {character['specific_actions']['pose']}")
            print(f" Action: {character['specific_actions']['action']}")


        print("\n- Setting\n")
        print(f"Primary Location: {scene['setting']['primary_location']}")
        print(f"Secondary Location: {scene['setting']['secondary_location']}")
        print(f"Geographical Context: {scene['setting']['geographical_context']}")
        print(f"Geographical Location: {scene['setting']['geographical_location']}")
        print(f"Time of Day: {scene['setting']['time_of_day']}")
        print(f"Weather: {scene['setting']['weather']}")
        print(f"Key Location Elements: {', '.join(scene['setting']['key_location_elements'])}")

        print("\n- Visual Elements\n")
        print(f"Camera Angle: {scene['visual_elements']['camera_angle']}")
        print(f"Cultural Artifacts: {', '.join(scene['visual_elements']['cultural_artifacts'])}")
        print(f"Financial Symbols: {', '.join(scene['visual_elements']['financial_symbols'])}")

        print(f"\n- Mood: {scene['mood']}")

        print(f"\nDescription: {scene['description']}")
    print()

# Scene Abstraction

In [ ]:
# print abstract information about which characters are present in which scenes
characters_list = []
for profile in final_character_profile_gen['character_profiles']:
  character = {}
  character['character_number'] = profile['character_number']
  character['character_name'] = profile['name']
  character['character_role'] = profile['character_role']

  characters_list.append(character)

def get_character_scene_mapping(json_data):
    character_scene_mapping = defaultdict(list)
    for module in json_data["modules"]:
        for scene in module["scenes"]:
            scene_title = scene["title"]
            for character in scene["characters_present"]:
                character_name = character["name"]
                character_scene_mapping[character_name].append(scene_title)
    result = [
        {"character_name": name, "scenes_present": titles}
        for name, titles in character_scene_mapping.items()
    ]

    return result

def add_scenes_to_character_abstract(characters_list, character_scene_mapping):
    scene_mapping_dict = {entry["character_name"]: entry["scenes_present"] for entry in character_scene_mapping}

    for character in characters_list:
        character_name = character.get("character_name")
        character["scenes_present"] = scene_mapping_dict.get(character_name, [])

    return characters_list

character_scene_mapping = get_character_scene_mapping(previous_scenes)

character_abstract = add_scenes_to_character_abstract(characters_list, character_scene_mapping)

for character in character_abstract:
  print(character, end='\n\n')